# Read Table
#### 1.
It is necessary to import the libraries, then the wikitable is read by BeautifulSoup library
#### 2.
Each Row is load to matrix, then turn to a Pandas DataFrame, and clean
#### Assumptions
The three Columns has to be in the same order ['PostalCode', 'Borough',  'Neighborhood']  


In [257]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#read the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})


In [317]:
matrix=[]
for tr in table.find_all("tr"):
    td=tr.find_all('td')
    matrix.append([x.text.strip() for x in td])
#     print(td)
columns = ['PostalCode', 'Borough',  'Neighborhood']

df = pd.DataFrame(matrix, columns = columns)
df = df[1:]
df = df.loc[df[columns[1]] != "Not assigned"]
df.loc[df[columns[2]] == "Not assigned",columns[2]] = df.loc[df[columns[2]]=="Not assigned",columns[1]]
df = df.groupby([columns[0],columns[1]])[columns[2]].apply(lambda tags: ','.join(tags)).to_frame().reset_index()
df.shape

(103, 3)

In [318]:
latlng = pd.read_csv('https://cocl.us/Geospatial_data')
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [323]:
dfLoc = df.sort_values('PostalCode')
latlng.sort_values('Postal Code', inplace = True)
dfLoc['Latitude'] = latlng['Latitude']
dfLoc['Longitude'] = latlng['Longitude']
dfLoc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
